In [ ]:
from pycalphad import Model, Database, Workspace, variables as v
from pycalphad.property_framework.metaproperties import IsolatedPhase

import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as spi
from symbolic_hulls_func_aggr import *
import scipy.optimize as opt

from sympy import symbols, preorder_traversal
import sympy as sp

import sympy as sp
import numpy as np
from scipy import integrate as spi
import plotly.graph_objects as go


## Pull out the data in numerical form

In [ ]:
db = Database("../TDDatabaseFiles_temp/alzn_mey.tdb")
comps = ['AL', 'ZN']

wks2 = Workspace(db, comps,
                ['FCC_A1', 'HCP_A3', 'LIQUID'],
                {v.X('ZN'):(0,1,0.02), v.T: 600, v.P:101325, v.N: 1})

fig = plt.figure()
ax = fig.add_subplot()

x = wks2.get(v.X('ZN'))
ax.set_xlabel(f"{v.X('ZN').display_name} [{v.X('ZN').display_units}]")

for phase_name in wks2.phases:
    metastable_wks = wks2.copy()
    metastable_wks.phases = [phase_name]
    prop = IsolatedPhase(phase_name, metastable_wks)(f'GM({phase_name})')
    prop.display_name = f'GM({phase_name})'
    ax.plot(x, wks2.get(prop), label=prop.display_name)
ax.legend()

plt.show()

## Extract the explicit energy functions for two phases and use them to compute energy values

In [ ]:
# Now we are going to extract two energy functions in empirical form
model1 = Model(db, comps, 'HCP_A3')
model2 = Model(db, comps, 'FCC_A1')

expr1 = sp.sympify(model1.GM)
expr2 = sp.sympify(model2.GM)

def get_symbols(expr):
    ordered_symbols = []
    seen = set()

    for node in preorder_traversal(expr):
        if node.is_Symbol and node not in seen:
            ordered_symbols.append(node)
            seen.add(node)

    return ordered_symbols

expr1_symbols = get_symbols(expr1)
expr2_symbols = get_symbols(expr2)

# Evaluate the function at some temperature
temperature = 600
expr1_func = expr1.subs(v.T, temperature)
expr2_func = expr2.subs(v.T, temperature)

# Replace the Al fraction with 1-Zn fraction
expr1_func = expr1_func.subs(expr1_symbols[0], 1-expr1_symbols[1]).evalf()
expr2_func = expr2_func.subs(expr2_symbols[0], 1-expr2_symbols[1]).evalf()

# Plot from 0 to 1 on the composition axis
x_range = np.arange(0, 1, 0.01)
expr1_values = [float(expr1_func.subs(expr1_symbols[1], x_val)) for x_val in x_range]
expr2_values = [float(expr2_func.subs(expr2_symbols[1], x_val)) for x_val in x_range]

# Display the functions
display(expr1_func)
display(expr2_func)

# Plot
plt.plot(x_range, expr1_values, label='GM(HCP_A3)')
plt.plot(x_range, expr2_values, label='GM(FCC_A1)')
plt.xlabel(f"{v.X('ZN').display_name} [{v.X('ZN').display_units}]")
plt.legend()
plt.show()

In [ ]:
# To use my method on these functions, we need to fit polynomials to the splines. We can do this over the composition range using Galerkin's method
def fit_polynomial_with_Galerkin(f_peicewise, highest_order, domain):
    highest_order += 1

    var = f_peicewise.free_symbols.pop()
    basis = [var**i for i in range(highest_order)]

    M = sp.zeros(highest_order, highest_order)
    for i in range(highest_order):
        for j in range(highest_order):
            lambda_func = sp.lambdify(var, basis[i]*basis[j], 'numpy')
            M[i,j] =  spi.quad(lambda_func, domain[0], domain[1])[0]
            
    b_vec = sp.zeros(highest_order,1)
    for i in range(highest_order):
        # We need to make the product function numerical to integrate
        lambda_func = sp.lambdify(var, basis[i]*f_peicewise, 'numpy')
        b_vec[i,0] = spi.quad(lambda_func, domain[0], domain[1])[0] # Take only the value and drop the error term 

    c = M.LUsolve(b_vec)

    return sp.simplify(sum(c[i,0]*basis[i] for i in range(highest_order)))

# Fit the polynomials
polynomial_degree = 4
P1 = fit_polynomial_with_Galerkin(expr1_func, polynomial_degree, (0, 1))
P2 = fit_polynomial_with_Galerkin(expr2_func, polynomial_degree, (0, 1))

# Round off the polynomials
P1 = P1.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))
P2 = P2.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))

# Plot both polynomials against the splines
P1_lambda = sp.lambdify(P1.free_symbols.pop(), P1, 'numpy')
P2_lambda = sp.lambdify(P2.free_symbols.pop(), P2, 'numpy')

expr1_lambda = sp.lambdify(expr1_func.free_symbols.pop(), expr1_func, 'numpy')
expr2_lambda = sp.lambdify(expr2_func.free_symbols.pop(), expr2_func, 'numpy')

# Extend the range slightly beyond [0,1]
x_vals = np.linspace(-1, 2, 400)

P1_lambda_vals = P1_lambda(x_vals)
P2_lambda_vals = P2_lambda(x_vals)

expr1_lambda_vals = expr1_lambda(x_vals)
expr2_lambda_vals = expr2_lambda(x_vals)

plt.figure(figsize=(8, 5))
# Here are the functions from calphad
plt.plot(x_vals, expr1_lambda_vals, label="HCP_A3", linestyle="--", color="b")
plt.plot(x_vals, expr2_lambda_vals, label="FCC_A1", linestyle="--", color="g")
# Here are the polynomials
plt.plot(x_vals, P1_lambda_vals, label="Polynomial HCP_A3", color="b", alpha = .7)
plt.plot(x_vals, P2_lambda_vals, label="Polynomial FCC_A1", color="g", alpha = .7)
# These show valid composition ranges
plt.axvline(0, linestyle="--", color="gray", lw=2)
plt.axvline(1, linestyle="--", color="gray", lw=2)
# Add the labels
plt.xlabel(f"{v.X('ZN').display_name} [{v.X('ZN').display_units}]")
plt.ylabel("G")
plt.title("Least Squares Polynomial Approximators of Energy Functions")
# Here is a good range to look at
plt.xlim(-1, 2)
plt.ylim(-30000, -13000)

plt.legend()
plt.grid(True)
plt.show()

## Now we will calculate the hyperplane boundary on the curves with the discriminant method

In [ ]:
# make both polynomials have the same variable
y = symbols("y")
P1 = P1.subs(P1.free_symbols.pop(), y)
P2 = P2.subs(P2.free_symbols.pop(), y)

display(P1)
display(P2)

boundary1 = hpboundry_f2_to_f1(P1, P2)
boundary2 = hpboundry_f2_to_f1(P2, P1)

boundary1_lambda = sp.lambdify(y, boundary1, 'numpy')
boundary1_prime_lambda = sp.lambdify(y, sp.diff(boundary1), 'numpy')

boundary2_lambda = sp.lambdify(y, boundary2, 'numpy')
boundary2_prime_lambda = sp.lambdify(y, sp.diff(boundary2), 'numpy')

root1 = opt.newton(boundary1_lambda, x0=.1, fprime=boundary1_prime_lambda)
root2 = opt.newton(boundary2_lambda, x0=0, fprime=boundary2_prime_lambda)

# Here we define the line through the points we calculated
m = (P2_lambda(root2) - P1_lambda(root1)) / (root2 - root1)
b = P1_lambda(root1) - m * root1
# Extend the range slightly beyond [0,1]
x_vals = np.linspace(-1, 2, 400)

plt.figure(figsize=(8, 5))
# Here are the polynomials
plt.plot(x_vals, P1_lambda_vals, label="Polynomial HCP_A3", color="b", alpha = .7)
plt.plot(x_vals, P2_lambda_vals, label="Polynomial FCC_A1", color="g", alpha = .7)
# Here are the tangency points and line
plt.scatter([root1, root2], [P1_lambda(root1), P2_lambda(root2)], label="Tangency Points", color="k")
plt.plot(x_vals, m*x_vals + b, label="Hyperplane", color="r", alpha = .7)
# These show valid composition ranges
plt.axvline(0, linestyle="--", color="gray", lw=2)
plt.axvline(1, linestyle="--", color="gray", lw=2)
# Add the labels
plt.xlabel(f"{v.X('ZN').display_name} [{v.X('ZN').display_units}]")
plt.ylabel("G")
plt.title("Least Squares Polynomial Approximators of Energy Functions")
# Here is a good range to look at
plt.xlim(-1, 2)
plt.ylim(-30000, -13000)

plt.legend()
plt.grid(True)
plt.show()